# TSDC - Sliding window + klasifikator znakova (KNM)

---

## Motivacija

Konvolutivne neuronske mreže pokazuju izuzetno dobre rezultate u klasifikaciji podataka, posebno u oblasti klasifikacije objekata na slikama. Obučavanje na dovoljno velikom skupu podataka koje je vrlo često veoma dugotrajno, rezultuje klasifikatorom koji je pretežno robustan, u zavisnosti od podataka koji su pohranjeni.

Činjenicu da je moguće obučiti efikasan klasifikator idejno je moguće upariti sa problemom detekcije objekta na sceni. Osnovna ideja je segmentacija slike, odnosno pravljenje klizajućeg prozora koji bi segmente slike pružao kao ulaz u klasifikator, a u zavisnosti od izlaza bi se utvrdilo da li se u tom regionu nalazi objekat od interesa.

<h2><center>Predloženi tok algoritma</center></h2>

![Tok algoritma](images/algoritam.png)

### Algoritam

1. Učitavanje slike od interesa
2. Učitavanje neuronske mreže
3. Obrada slike tako da odgovara očekivanom ulazu u NM
4. Zadavanje veličine klizajućeg prozora
5. Zadavanje pozicije prozora
6. Ekstrakcija podataka iz prozora
7. Prilagođavanje veličine ulaza za NM
8. Obrada podataka u NM
9. Provera pouzdanosti predikcije i klase
10. Ponoviti korak 5. do završetka iteracije
11. Ponoviti od koraka 4. za različitu veličinu prozora


---

## Neuronska mreža


<img alt="Model neuronske mreže" style="float: right;" src="images/model.png" width="30%">

Zadatak neuronske mreže je klasifikacija slika veličine 48x48 u jednu od 43 kategorije saobraćajnih znakova. Skup podataka u vidu 50000 saobraćajnih znakova koji su upotrebljeni za obučavanje i validaciju deo je skupa **GTSRB** i sastoji se od već izdvojenih slika znakova koji su već klasifikovani, kao i određenog broja neklasifikovanih znakova za validaciju.

Kao platforma za razvoj neuronske mreže upotrebljena je *Theano* i *Keras* biblioteka, dok se za obradu slike upotrebljava *Scikit* zbirka biblioteka, u programskom jeziku *Python*.

Postavljen je model duboke neuronske mreže sa konvolutivnim slojevima. **Model neuronske mreže nalazi se na slici, desno**. Radi minimizovanja funkcije cilja korišćena je optimizacija stohastičkim opadajućim gradijentom, brzina obučavanja 0.01, *decay* 10<sup>-6</sup>, a momenat 0.9.

Skup podataka za obučavanje je obrađen tako što je slikama promenjena veličina na 48x48 i prevedene su iz RGB domena u *grayscale* domen. *Kako bi testiranje i upotreba mreže bila uspešna, potrebno je obezbediti da su ulazni podaci uvek u ovom formatu*.

Validacija je izvršena na osnovu testnog skupa i provere da li je mreža klasifikovala znak u prethodno anotiranu kategoriju. U zavisnosti od epoha, pouzdanost klasifikacije varirala je od 84.62% za jednu epohu, do 97.47% za dvadeset epoha. Kako bi se umanjila verovatnoća overfitovanja mreže, podaci su nasumično pomešani pre faze obučavanja. Confusion matrice upotrebljene su kako bi se utvrdila pogrešno klasifikovana kategorija, a dat je pregled za matrice nakon jedne, odnosno 20 epoha:

<img alt="Confusion matrices" src="images/confusion.png" width="60%">

Inspekcijom pogrešno klasifikovanih znakova može se doći do zaključka da se radi o znakovima koji su delimično zaklonjeni ili gde je kvalitet slike značajno narušen tako da ni ljudskim zaključivanjem nije moguće odrediti klasu znaka. Neki od primera pogrešno klasifikovanih znakova su ilustrovani: 

<img alt="Missed images" src="images/missed.png" width="55%">


Na osnovu validacije na 12630 slika moguće je zaključiti da je neuronska mreža uspešno obučena za detekciju saobraćajnih znakova definisanih obučavajućim skupom.

Dodatne informacije i objašnjenje moguće je videti i na [posteru](vs_cnn.pdf), gde su performanse upoređene sa pristupom klasifikaciji upotrebom Support Vector Mašine (SVM) na ekstrahovanim osobinama slike u vidu histograma orijentisanih vrednosti (HOG).

---

## Sliding window i detekcija

Nakon učitavanja ulazne slike neophodno je skenirati svaki region slike u potrazi za objektima od interesa. Sliding window se postavlja sa ranije zadatim parametrima o visini i širini i skenira se cela slika. Nakon završenog procesa skeniranja, veličina prozora se može povećati u određenim inkrementalnim koracima kako bi se ostvarila potencijalna detekcija koja će učiniti invarijantnom veličinu objekta.

Prilikom svake iteracije, učitava se region koji je obuhvaćen prozorom i on predstavlja ulazak u neuronsku mrežu - klasifikator. Neophodno je izvršiti transformaciju na prethodno učitanim podacima tako da odgovaraju očekivanom ulazu u mrežu, pre svega u vidu skaliranja na odgovarajuće dimenzije, a zatim i transformacijom podataka kako bi se dobila veća preciznost pri klasifikaciji.

S obzirom na to da se radi o klasifikatoru koji se obučava na pozitivnim rezultatima, ishod klasifikacije će dati neku od klasa koja je detektovana. Kako bi se izbegla takva lažna detekcija, postavlja se korak pouzdanosti detekcije koji bi trebalo da u slučaju pogađanja sa niskom sigurnošću, takve detekcije odbaci.

---

## Diskusija

### Performanse

Opisani algoritam zahteva veliki broj klasifikacija, koji zavisi od veličine slike, veličine prozora za detekciju, koraka pomeraja prozora kao i eventualne opcije da se pretraga ponovi sa modifikovanom veličinom prozora.

Algoritam ne zadovoljava real-time kriterijum i veoma je zahtevan po pitanju resursa.

### Pouzdanost

Pouzdanost ovog metoda je veoma mala. Najveći problem koji se javlja jeste nemogućnost utvrđivanja da li je došlo do lažnog pogotka, odnosno da li je klasifikator detektovao objekat na mestu gde ne bi trebalo. Ovaj problem nije moguće prevazići podešavanjem parametara, pre svega iz razloga zato što je neuronska mreža trenirana na pozitivnom skupu podataka. Javlja se klasični *garbage in - garbage out* problem. Nije moguće unapred predvideti koji će ulaz biti doveden na neuronsku mrežu - a izlaz će sigurno biti neka od klasa znakova. Procenat sigurnosti klasifikacije ne može biti pouzdan pokazatelj da je region zaista objekat od interesa. Dodatni problemi javljaju se u slučaju znakova koji su suviše mali ili suviše veliki, ukoliko sliding window ne obuhvati region slike na dovoljno dobar način, nije moguće detektovati znak.

Mnogobrojne lažne detekcije, kao i problem veličine prozora za detekciju moguće je videti na primerima:


![Slika 1](images/im1.png)
![Slika 2](images/im2.png)
![Slika 2](images/im3.png)


---

## Zaključak

Pristup koji je ispitivan nije adekvatan za rešavanje navedenog problema. Neuronska mreža i pored svoje veoma visoke pouzdanosti u zadatku klasifikacije znakova može da radi samo na ograničenom skupu zadataka - klasifikaciji slika znakova u 43 kategorije. Dovođenjem nepoznatog ulaza nije moguće garantovati ishod ove vrste klasifikatora. 

Predlog rešenja koji će se istraživati jeste obučavanje klasifikatora na osnovu pozitivnih i negativnih primera, tako da na osnovu karakteristika klasifikator može binarno da odluči da li region sadrži objekat od interesa ili ne.